### Student Information
Name: 蕭靖澂 Ching-Cheng Hsiao

Student ID: 110033632

GitHub ID: hsiaooo

Kaggle name:

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2021-Lab2-master Repo](https://github.com/fhcalderon87/DM2021-Lab2-master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/c/dm2021-lab2-hw2/) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 24th 11:59 pm, Friday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 29th 11:59 pm, Wednesday)__. 

In [1]:
### Begin Assignment Here

### Load kaggle dataset

In [2]:
# load the libraries
import os
import json
import pandas as pd
import numpy as np

# load the json
path       = os.path.dirname(os.path.abspath("tweets_DM.json"))
# raw_data = pd.read_json(path+"/dataset/tweets_DM.json",lines=True)
# tweets   = pd.json_normalize(data=raw_data['_source'])
# identify = pd.read_csv(basePath+"/dataset/data_identification.csv")
# emotion  = pd.read_csv(basePath+"/dataset/emotion.csv")

In [3]:
# tweets.head()

In [4]:
# # rename column names
# tweets = tweets.rename(index=str, columns={"tweet.text":"text", "tweet.tweet_id":"tweet_id", "tweet.hashtags":"hashtags"})

# # add identify tags to dataframe
# tweets = pd.merge(tweets, identify, on="tweet_id")

# tweets.head()

In [5]:
# # get training set and test set
# train_df = tweets[tweets["identification"] == "train"]
# test_df  = tweets[tweets["identification"] == "test"]

# # drop identification tags
# train_df.drop(columns=["identification"], inplace=True)
# test_df.drop(columns=["identification"], inplace=True)

# print(f'Training set\n{train_df.head()}\n')
# print(f'Testing set\n{test_df.head()}')

In [6]:
# add emotion column
# train_df = pd.merge(train_df, emotion, on="tweet_id")
# test_df = test_df.assign(emotion="")

In [7]:
# use tweet_id as index
# train_df.set_index("tweet_id",inplace=True)
# test_df.set_index("tweet_id",inplace=True)

# train_df.head()

In [8]:
# save to pickle file
# train_df.to_pickle("train_df.pkl")
# test_df.to_pickle("test_df.pkl")

In [9]:
# load a pickle file
train_df = pd.read_pickle(path+"/train_df.pkl")
test_df = pd.read_pickle(path+"/test_df.pkl")

### Pre-processing

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer(preserve_case=False)
tfidf = TfidfVectorizer(max_features=20000, stop_words='english', tokenizer=tknzr.tokenize)

# fitting
tfidf.fit(train_df['text'])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_features=20000, stop_words='english',
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f725630b880>>)

In [11]:
# transforming training sets
X_train = tfidf.transform(train_df['text'])
X_train.shape

(1455563, 20000)

In [12]:
# transforming testing sets
X_test = tfidf.transform(test_df['text'])
X_test.shape

(411972, 20000)

In [13]:
# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [14]:
print(y_test[0:4])

tweet_id
0x28b412    
0x2de201    
0x218443    
0x2939d5    
Name: emotion, dtype: object


#### Logistic regression

In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=6,n_jobs=-1,max_iter=1000)
lr.fit(X_train,y_train)

LogisticRegression(C=6, max_iter=1000, n_jobs=-1)

In [16]:
pred_result_lr = lr.predict(X_test)
pred_result_lr.shape

(411972,)

In [17]:
# save the result
test_df['emotion']=pred_result_lr
test_df.drop(columns=['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/lr_tfidf.csv')

#### XGBoost Regressor

In [18]:
emotion = pd.Categorical(y_train.iloc[:], categories=y_train.iloc[:].unique(), ordered=True)

y_train.iloc[:] = emotion.codes
print(y_train)
print('anger \t\t= ', y_train[9])
print('anticipation \t= ', y_train[0])
print('disgust \t= ', y_train[18])
print('fear \t\t= ', y_train[2])
print('joy \t\t= ', y_train[3])
print('sadness \t= ', y_train[1])
print('surprise \t= ', y_train[19])
print('trust \t\t= ', y_train[15])

tweet_id
0x376b20    0
0x2d5350    1
0x1cd5b0    2
0x1d755c    3
0x2c91a8    0
           ..
0x321566    3
0x38959e    3
0x2cbca6    3
0x24faed    3
0x34be8c    3
Name: emotion, Length: 1455563, dtype: object
anger 		=  4
anticipation 	=  0
disgust 	=  6
fear 		=  2
joy 		=  3
sadness 	=  1
surprise 	=  7
trust 		=  5


In [19]:
import xgboost as xgb

xgbc = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgbc.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, ...)

In [20]:
pred_result_xgbc = xgbc.predict(X_test)

pred_result_xgbc.shape
print(pred_result_xgbc)

[3 0 1 ... 3 3 3]


In [21]:
print(pred_result_xgbc[-10:])

[3 3 3 3 3 3 0 3 3 3]


In [22]:
pred_result = []
for i, emo in enumerate(pred_result_xgbc):
    if emo == 0:
        pred_result.append('anticipation')
    elif emo == 1:
        pred_result.append('sadness')
    elif emo == 2:
        pred_result.append('fear')
    elif emo == 3:
        pred_result.append('joy')
    elif emo == 4:
        pred_result.append('anger')
    elif emo == 5:
        pred_result.append('trust')
    elif emo == 6:
        pred_result.append('fear')
    elif emo == 7:
        pred_result.append('surprise')

In [23]:
pred = np.array(pred_result)
print(pred)

['joy' 'anticipation' 'sadness' ... 'joy' 'joy' 'joy']


In [24]:
# save the result
test_df['emotion'] = pred
test_df.drop(columns = ['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns = ['emotion']
test_df.to_csv(path+'/submission/xgbc_tfidf.csv')
print('Saved file!')

KeyError: "['hashtags' 'text'] not found in axis"

### LSTM

In [ ]:
from keras.models import Model
from keras.utils import np_utils
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import rmsprop_v2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

In [ ]:
# load a pickle file
train_df = pd.read_pickle(path+"/train_df.pkl")
test_df = pd.read_pickle(path+"/test_df.pkl")

# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [ ]:
max_words = 20000
max_len = 300
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train_df['text'])

In [ ]:
train_seq = tok.texts_to_sequences(train_df['text'])
test_seq = tok.texts_to_sequences(test_df['text'])

train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)

print(train_seq_mat.shape)
print(test_seq_mat.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
# y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

In [ ]:
train_emotion = pd.Categorical(y_train.iloc[:], categories=y_train.iloc[:].unique(), ordered=True)
test_emotion = pd.Categorical(y_test.iloc[:], categories=y_test.iloc[:].unique(), ordered=True)

y_train.iloc[:] = train_emotion.codes
y_test.iloc[:] = test_emotion.codes

print(y_train)
print(y_test)

In [ ]:
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

In [ ]:
inputs = Input(name='inputs',shape=[max_len])
## Embedding
layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
layer = LSTM(128)(layer)
layer = Dense(128,activation="relu",name="FC1")(layer)
layer = Dropout(0.5)(layer)
layer = Dense(output_shape,activation="softmax",name="FC2")(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss="categorical_crossentropy",optimizer=rmsprop_v2.RMSprop(),metrics=["accuracy"])

In [ ]:
model_fit = model.fit(train_seq_mat, y_train,batch_size=128, epochs=3, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] )

In [ ]:
# predict the result using our model
pred_result_lstm = label_decode(label_encoder, model.predict(test_seq_mat, batch_size=128))
pred_result_lstm[:5]

In [ ]:
# save the result
test_df['emotion']=pred_result_lstm
test_df.drop(columns=['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/keras_tfidf.csv')
print('Saved file')